# 가격, 전원, 탄소 계산에 필요한 정보
1. Instance Cost (Hourly, Monthly)
1. Storage Cost (Hourly, Monthly)
1. OS
1. 
- 계산을 어떻게 할 지, 계산에 필요한 정보들을 파악하고 수집하는 방법 모색해야 함

In [10]:
from typing import Dict
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Cost(BaseModel):
    # monthly: str = Field(description="Estimated monthly measurements")
    hourly: str = Field(description="Estimated measurements per hour")

class Instance(BaseModel):
    cloud_provider: str = Field(description="Name of cloud provider (GCP or AWS or Azure)")
    name: str = Field(description="Name of instance type")
    cpu: int = Field(description="The number of instance cpu cores")
    ram: float = Field(description="The capacity of instance ram (GiB)")
    storage: int = Field(description="The capacity of instance storage (GiB)")
    gpu: str = Field(default=None, description="Name of gpu model and memory (GiB) of instance")
    region: str = Field(description="Region of instance")
    cost: Cost = Field(description="Monthly and Hourly cost of instance")

class Estimate(BaseModel):
    instance: Instance = Field(description="Instance information of Cloud Provider")
    # pricing: Cost = Field(description="Estimated pricing while running an instance of the instance_type (USD)")
    power_consumption: Cost = Field(description="Estimated power consumption while running an instance of the instance_type(kWh)")
    carbon_footprint: Cost = Field(description="Estimated carbon footprint while running an instance of the instance_type(kg CO2)")
    description: str = Field(description="A rationale and detailed explanation for estimations")
    

class Result(BaseModel):
    gcp: Estimate = Field(description="Estimated Result of Google Cloud Platform(GCP)")
    aws: Estimate = Field(description="Estimated Result of Amazon Web Services(AWS)")
    azure: Estimate = Field(description="Estimated Result of Microsoft Azure")
    # conclusion: Estimate = Field(description="The most appropriate among gcp, aws, and azure")
    language_ratio: Dict[str, int] = Field(description="The key value is the programming language used and the value is the number of bytes in which the programming language is used.")

output_parser = JsonOutputParser(pydantic_object=Result)
# output_parser = JsonOutputParser()

In [14]:
from typing import Dict
from google.cloud.firestore_v1.base_query import FieldFilter, Or, And
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

def get_latest_price(instance: Instance) -> Instance:
    """Based on gemini's suggestion, get the cheapest AWS instance with latest information"""
    if not firebase_admin._apps:
        cred = credentials.Certificate('firebase-svc-account-key.json')
        app = firebase_admin.initialize_app(cred)
    db = firestore.client()
    ref = db.collection("cloud_cost")
    vendor_filter = FieldFilter("vendor", "==", instance.cloud_provider)
    name_filter = FieldFilter("name", "==", instance.name)
    cpu_filter = FieldFilter("cpu", "==", instance.cpu)
    ram_filter = FieldFilter("ram", "==", instance.ram)
    resource_filter = And(filters=[cpu_filter, ram_filter])
    instance_filter = Or(filters=[name_filter, resource_filter])
    final_filter = And(filters=[vendor_filter, instance_filter])
    docs = ref.where(filter=final_filter).stream()

    lowest_instance = {"cost_per_hour": float("inf")}
    for doc in docs:
        if lowest_instance["cost_per_hour"] > doc.to_dict()["cost_per_hour"]:
            lowest_instance = doc.to_dict()
    instance = Instance(
        cloud_provider=instance.cloud_provider,
        name=lowest_instance["name"], 
        cpu=lowest_instance["cpu"], 
        ram=lowest_instance["ram"], 
        storage=instance.storage,
        gpu=lowest_instance["gpu"],
        region=lowest_instance["region"],
        cost=Cost(hourly=lowest_instance["cost_per_hour"])
    )
    return instance

In [16]:
gemini_result = Result(**{
    'gcp': {
        'instance': {
            'cloud_provider': 'GCP',
            'name': 'e2-micro',
            'cpu': '2',
            'ram': 0.6,
            'storage': 30,
            'gpu': 'None',
            'region': 'us-central1',
            'cost': {'monthly': '$13.04', 'hourly': '$0.006'}
        },
        'power_consumption': {'monthly': '9.78 kWh', 'hourly': '0.004 kWh'},
        'carbon_footprint': {'monthly': '4.401 kg CO2', 'hourly': '0.002 kg CO2'},
        'description': "This Kotlin Spring Boot application, connecting to a MySQL database, can be run on a minimal GCP e2-micro instance. The e2-micro, with 2 vCPUs and 0.6 GB memory, suffices for development and light traffic. We estimate 30GB storage for the application and database. The estimated cost is based on GCP's pricing, and power consumption is a conservative estimate. The carbon footprint is derived using US-central1's carbon efficiency."
    },
    'aws': {
        'instance': {
            'cloud_provider': 'AWS',
            'name': 't4g.nano',
            'cpu': '2',
            'ram': 0.5,
            'storage': 20,
            'gpu': 'None',
            'region': 'us-east-1',
            'cost': {'monthly': '$10.24', 'hourly': '$0.004'}
        },
        'power_consumption': {'monthly': '7.68 kWh', 'hourly': '0.003 kWh'},
        'carbon_footprint': {'monthly': '3.456 kg CO2', 'hourly': '0.001 kg CO2'},
        'description': "For AWS, a t4g.nano instance (2 vCPU, 0.5 GB memory) is suitable. 20GB storage is allocated for application and database files. Cost calculation is based on AWS pricing, and power consumption is a conservative estimate for this instance type. The carbon footprint is calculated using us-east-1's carbon efficiency data."
    },
    'azure': {
        'instance': {
            'cloud_provider': 'Azure',
            'name': 'B1s',
            'cpu': '1',
            'ram': 1,
            'storage': 30,
            'gpu': 'None',
            'region': 'eastus',
            'cost': {'monthly': '$13.34', 'hourly': '$0.005'}
        },
        'power_consumption': {'monthly': '9.99 kWh', 'hourly': '0.004 kWh'},
        'carbon_footprint': {'monthly': '4.496 kg CO2', 'hourly': '0.002 kg CO2'},
        'description': "Azure's B1s instance (1 vCPU, 1 GB memory) is a good fit. 30GB storage is allocated for the application and database. Cost estimation is based on Azure's pricing, and power consumption is a conservative estimate. The carbon footprint is derived using Azure's carbon emission data for the eastus region."
    },
    'language_ratio': {
        'Kotlin': 12215,
        'Properties': 1246,
        'YAML': 804,
        'JSON': 1782
    }
})

In [18]:
best_aws_instance = get_latest_price(gemini_result.aws.instance)
best_gcp_instance = get_latest_price(gemini_result.gcp.instance)

In [21]:
gemini_result.aws.instance = best_aws_instance
gemini_result.gcp.instance = best_gcp_instance

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Output
# Ensure the output is structured in a clear and detailed manner, adhering to the JSON format specified by the following guide: {format_instruction}
prompt = PromptTemplate(
    template="""
""",
    input_variables=[""],
    partial_variables={"format_instruction": output_parser.get_format_instructions()}
    )
chain = prompt | llm | output_parser
# chain = prompt | llm | StrOutputParser()